In [1]:
import numpy as np
rng = np.random.default_rng(0)

from glmnet.coxdev import right_censored
import rpy2
%load_ext rpy2.ipython

n = 2000
event_time = np.hstack([[1.2]*3, [2.5]*4, [1.1]*4, rng.standard_exponential(n-11)])
status = rng.choice([0, 1], size=n)
eta = rng.standard_normal(n)

In [2]:
rc = right_censored(event_time, status)
result = rc(eta)

In [3]:
%%R -i event_time,status,eta -o l,g,h
library(glmnet)
n = length(event_time)
library(survival)
y = data.frame(time=event_time, status=status)
y_s = Surv(event_time, status)
w = rep(1, n)
eta = as.numeric(eta)
l = glmnet:::coxnet.deviance2(pred=eta, y=y_s, std.weights=FALSE)
g = glmnet:::coxgrad2(eta, y_s, std.weights=FALSE, diag.hessian=TRUE)
h = attr(g, 'diag_hessian')

Loading required package: Matrix
Loaded glmnet 4.1-4


In [4]:
assert np.allclose(l, result.deviance)

In [5]:
assert np.allclose(2*g, result.grad)

In [6]:
assert np.allclose(2*h, result.diag_hessian)